### DATA PT14-extra - Proyecto final - Grupo 3
#### Control de calidad de los datos del archivo Google Maps review de los estados de California (CA) y Nevada(NV)

In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

Comenzamos con California

In [4]:
# Obtener la lista de archivos JSON en carpeta
archivos_CA = glob.glob(os.path.join(r'C:\Users\gonza\OneDrive\Desktop\Proyecto Grupal Henry\Maps\review-California', "*.json"))

In [6]:
# Se guardará archivos JSON en formato pickle (binario) el cual es más rápido para futuras consultas
for archivo in archivos_CA:
    df = pd.read_json(archivo, lines=True)
    df.to_pickle(archivo.replace(".json", ".pkl"))  # Guardar como .pkl

In [7]:
# Carga los archivos Pickle generados y los combina en un solo DataFrame.
df_list = [pd.read_pickle(archivo.replace(".json", ".pkl")) for archivo in archivos_CA] # convierte el nombre de cada archivo de .json a .pkl para cargarlo correctamente.
df_rev_CA = pd.concat(df_list, ignore_index=True)

df_rev_CA.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [8]:
df_rev_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB


In [9]:
# Eliminamos, por ahora, pics y resp. no usaremos esas columnas
df_rev_CA = df_rev_CA.drop(columns=['pics', 'resp'])

In [10]:
# # reviso si hay reviews duplicadas
df_dup = df_rev_CA[df_rev_CA.duplicated(subset=['user_id', 'gmap_id', 'time'], keep=False)]
df_dup = df_dup.sort_values(['user_id', 'gmap_id', 'time'])

print(df_dup)

              user_id            name           time  rating  \
1028819  1.000003e+20  Daisy Villalon  1559520478805       3   
1028824  1.000003e+20  Daisy Villalon  1559520478805       3   
467997   1.000009e+20    Kyle Cassara  1552753057554       2   
468004   1.000009e+20    Kyle Cassara  1552753057554       2   
13055    1.000011e+20       Dan Serns  1630954898274       5   
...               ...             ...            ...     ...   
1077032  1.184459e+20  Jonathan Higgs  1542656249878       5   
1235163  1.184461e+20  Scott Mortimer  1516986179854       4   
1235170  1.184461e+20  Scott Mortimer  1516986179854       4   
920713   1.184465e+20     Jack Evanko  1489942134459       4   
920727   1.184465e+20     Jack Evanko  1489942134459       4   

                                                      text  \
1028819  (Translated by Google) Good food but service m...   
1028824  (Translated by Google) Good food but service m...   
467997   I don't know why but they come aroun

Hay 149377 reviews duplicadas por las columnas 'user_id', 'gmap_id', 'time' para el estado de CA. Las elimino

In [11]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_CA_sindup = df_rev_CA.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [12]:
# rename column name to user_name
df_rev_CA_sindup.rename(columns={'name': 'user_name'}, inplace=True)

In [13]:
# convertir columna time en int64 a datetime
df_rev_CA_sindup['date'] = pd.to_datetime(df_rev_CA_sindup['time'], unit='ms')

In [14]:
# Eliminamos la columan time
df_rev_CA_sindup = df_rev_CA_sindup.drop(columns=['time'])
df_rev_CA_sindup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2624746 entries, 0 to 2699999
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    float64       
 1   user_name  object        
 2   rating     int64         
 3   text       object        
 4   gmap_id    object        
 5   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 140.2+ MB


In [15]:
# guardo df_rev_NJ_sindup
df_rev_CA_sindup.to_parquet(r'C:\Users\gonza\OneDrive\Desktop\Proyecto Grupal Henry\Maps\df_rev_CA_sindup.parquet' , engine='fastparquet')

In [17]:
# leo el archivo de las pizzerias mt_pizza_NJNY para hacer join con este
# How to read a Parquet file into Pandas DataFrame?
df_mt_pizza_CANV = pd.read_parquet(r'C:\Users\gonza\OneDrive\Desktop\Proyecto Grupal Henry\Maps\df_mt_pizza_CANV_sindup_verif_coord.parquet' , engine='fastparquet')

In [18]:
df_mt_pizza_CANV.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2395 entries, 6738 to 3023082
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         2395 non-null   object 
 1   business_name   2395 non-null   object 
 2   address         2387 non-null   object 
 3   city            2395 non-null   object 
 4   state           2395 non-null   object 
 5   zip_code        2395 non-null   object 
 6   latitude        2395 non-null   float64
 7   longitude       2395 non-null   float64
 8   avg_rating      2395 non-null   float64
 9   num_of_reviews  2395 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 205.8+ KB


In [20]:
df_mt_pizza_CANV.to_excel(r'C:\Users\gonza\OneDrive\Desktop\Proyecto Grupal Henry\Maps\df_mt_pizza_CANV.xlsx')

In [21]:
# cuento la cantidad de pizzerias en cada estado / no puedo contar con una sola columna
df_mt_pizza_CANV[['state', 'business_name' ]].groupby('state').count()

,business_name
state,
CA,2087
NV,200
United,108


Googlemaps tiene datos de 2087 pizzerias en CA y 200 pizzerias en NV

In [22]:
# separo las pizzerias de CA de las de NV. Luego hago left join con las reviews y vuelvo a unir los dataframe
df_mt_pizza_CA = df_mt_pizza_CANV[df_mt_pizza_CANV['state'] == 'CA']
df_mt_pizza_CA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2087 entries, 6738 to 3023082
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         2087 non-null   object 
 1   business_name   2087 non-null   object 
 2   address         2079 non-null   object 
 3   city            2087 non-null   object 
 4   state           2087 non-null   object 
 5   zip_code        2087 non-null   object 
 6   latitude        2087 non-null   float64
 7   longitude       2087 non-null   float64
 8   avg_rating      2087 non-null   float64
 9   num_of_reviews  2087 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 179.4+ KB


In [23]:
df_mt_pizza_NV = df_mt_pizza_CANV[df_mt_pizza_CANV['state'] == 'NV']
df_mt_pizza_NV.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 15388 to 3009501
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gmap_id         200 non-null    object 
 1   business_name   200 non-null    object 
 2   address         200 non-null    object 
 3   city            200 non-null    object 
 4   state           200 non-null    object 
 5   zip_code        200 non-null    object 
 6   latitude        200 non-null    float64
 7   longitude       200 non-null    float64
 8   avg_rating      200 non-null    float64
 9   num_of_reviews  200 non-null    int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 17.2+ KB


Uno el dataframe de datos de pizzerias de CA en un left join con el de los datos de reviews de CA para obtener todas las reviews de pizzerias de CA y agregarle el nombre del local. Mas adelante hare los mismo para los locales de NV. Y luego unire los dos dataframes

In [24]:
df_pizza_CA_rev = pd.merge(df_mt_pizza_CA, df_rev_CA_sindup, how='left', on='gmap_id') # tengo que hacer left join

In [25]:
df_pizza_CA_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29025 entries, 0 to 29024
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gmap_id         29025 non-null  object        
 1   business_name   29025 non-null  object        
 2   address         29004 non-null  object        
 3   city            29025 non-null  object        
 4   state           29025 non-null  object        
 5   zip_code        29025 non-null  object        
 6   latitude        29025 non-null  float64       
 7   longitude       29025 non-null  float64       
 8   avg_rating      29025 non-null  float64       
 9   num_of_reviews  29025 non-null  int64         
 10  user_id         27574 non-null  float64       
 11  user_name       27574 non-null  object        
 12  rating          27574 non-null  float64       
 13  text            16632 non-null  object        
 14  date            27574 non-null  datetime64[ns]
dtypes:

Hay 12393 pizzerias del CA que no tienen reviews en el archivo usado. Se eliminaran con las de los locales de NV que tampoco tengan reseñas. </br>

HAgo lo mismo con Nevada

In [26]:
# Obtener la lista de archivos JSON en carpeta
archivos_json = glob.glob(os.path.join(r'C:\Users\gonza\OneDrive\Desktop\Proyecto Grupal Henry\Maps\review-Nevada', "*.json"))

In [27]:
# Se guardará archivos JSON en formato pickle (binario) el cual es más rápido para futuras consultas
for archivo in archivos_json:
    df = pd.read_json(archivo, lines=True)
    df.to_pickle(archivo.replace(".json", ".pkl"))  # Guardar como .pkl

In [28]:
# Carga los archivos Pickle generados y los combina en un solo DataFrame.
df_list = [pd.read_pickle(archivo.replace(".json", ".pkl")) for archivo in archivos_json] # convierte el nombre de cada archivo de .json a .pkl para cargarlo correctamente.
df_rev_NV = pd.concat(df_list, ignore_index=True)

df_rev_NV.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.028432e+20,CDL CO,1622740504476,5,No frills smaller coffee counter in the corner...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9
1,1.149544e+20,So_ Blind,1573159413820,1,Save yourself the frustration and just ignore ...,None,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9
2,1.165234e+20,Greg Hansen,1515251163362,3,I passed two Starbucks in order to get to this...,None,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9
3,1.033041e+20,Brett Powell,1506206762095,3,It does the job and is my preference over the ...,None,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9
4,1.141624e+20,M Aris,1468298771892,3,None,None,None,0x80c8c5cdced316b5:0xaf1ea0b1e42132a9


In [29]:
df_rev_NV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800000 entries, 0 to 1799999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 109.9+ MB


In [30]:
# # reviso si hay reviews duplicadas
df_dup_NV = df_rev_NV[df_rev_NV.duplicated(subset=['user_id', 'gmap_id', 'time'], keep=False)]
df_dup_NV = df_dup_NV.sort_values(['user_id', 'gmap_id', 'time'])

print(df_dup_NV)

             user_id              name           time  rating  \
742732  1.000006e+20        Susanne M.  1483491925016       2   
742751  1.000006e+20        Susanne M.  1483491925016       2   
728847  1.000026e+20     Adrian Tucker  1581247464740       5   
728991  1.000026e+20     Adrian Tucker  1581247464740       5   
668847  1.000034e+20     Daisy Tillman  1533009601759       5   
...              ...               ...            ...     ...   
1214    1.184459e+20     Menito Garcia  1599873059501       5   
549438  1.184460e+20      Mike Mendoza  1498327417640       5   
549439  1.184460e+20      Mike Mendoza  1498327417640       5   
734659  1.184464e+20  Rodrigo Abrantes  1573341243690       5   
734665  1.184464e+20  Rodrigo Abrantes  1573341243690       5   

                                                     text  pics  \
742732  Service was good, but the food...meh. Meats we...  None   
742751  Service was good, but the food...meh. Meats we...  None   
728847  Fun place 

Hay 47673 reseñas duplicadas para NV. Las elimino

In [31]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_NV_sindup = df_rev_NV.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [32]:
# convertir columna time en int64 a datetime
df_rev_NV_sindup['date'] = pd.to_datetime(df_rev_NV_sindup['time'], unit='ms')

In [33]:
# Eliminamos la columan time
df_rev_NV_sindup = df_rev_NV_sindup.drop(columns=['time'])
df_rev_NV_sindup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1775999 entries, 0 to 1799999
Data columns (total 8 columns):
 #   Column   Dtype         
---  ------   -----         
 0   user_id  float64       
 1   name     object        
 2   rating   int64         
 3   text     object        
 4   pics     object        
 5   resp     object        
 6   gmap_id  object        
 7   date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 121.9+ MB


In [34]:
# Eliminamos, por ahora, pics y resp. no usaremos esas columnas
df_rev_NV_sindup = df_rev_NV_sindup.drop(columns=['pics', 'resp'])

In [35]:
# rename column name to user_name
df_rev_NV_sindup.rename(columns={'name': 'user_name'}, inplace=True)

Uno el dataframe de datos de pizzerias de NV en un left join con el de los datos de reviews de NV para obtener todas las reviews de pizzerias de NV y agregarle el nombre del local.

In [36]:
df_pizza_NV_rev = pd.merge(df_mt_pizza_NV, df_rev_NV_sindup, how='left', on='gmap_id') # tengo que hacer left join

In [37]:
df_pizza_NV_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7396 entries, 0 to 7395
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gmap_id         7396 non-null   object        
 1   business_name   7396 non-null   object        
 2   address         7396 non-null   object        
 3   city            7396 non-null   object        
 4   state           7396 non-null   object        
 5   zip_code        7396 non-null   object        
 6   latitude        7396 non-null   float64       
 7   longitude       7396 non-null   float64       
 8   avg_rating      7396 non-null   float64       
 9   num_of_reviews  7396 non-null   int64         
 10  user_id         7331 non-null   float64       
 11  user_name       7331 non-null   object        
 12  rating          7331 non-null   float64       
 13  text            4761 non-null   object        
 14  date            7331 non-null   datetime64[ns]
dtypes: d

Hay 2635 pizzerias del NV de las que no hay reviews en el archivo usado.

In [38]:
# concateno las reviews de googlemaps de las pizzerias de NY y NJ
df_pizza_CANV_rev = pd.concat([df_pizza_CA_rev, df_pizza_NV_rev], ignore_index=True)

In [39]:
df_pizza_CANV_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36421 entries, 0 to 36420
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gmap_id         36421 non-null  object        
 1   business_name   36421 non-null  object        
 2   address         36400 non-null  object        
 3   city            36421 non-null  object        
 4   state           36421 non-null  object        
 5   zip_code        36421 non-null  object        
 6   latitude        36421 non-null  float64       
 7   longitude       36421 non-null  float64       
 8   avg_rating      36421 non-null  float64       
 9   num_of_reviews  36421 non-null  int64         
 10  user_id         34905 non-null  float64       
 11  user_name       34905 non-null  object        
 12  rating          34905 non-null  float64       
 13  text            21393 non-null  object        
 14  date            34905 non-null  datetime64[ns]
dtypes:

Hay 15028 pizzerias de las que no hay reviews en el archivo usado. Esto se infiere porque hay esa cantidad de user_id nulos. </br>
Es decir que al hacer un left join no habia una una fila con datos de review para ese local. </br>
Esas reseñas deben corresponder a un periodo anterior al del que tenemos reseñas.

Se eliminarán las filas correspondientes a esos locales de este dataframe. </br>
Se mantendrán las filas con los datos de esos locales en el dataframe de pizzerias ya que indican cantidad de reseñas y avg_rating. </br>

In [40]:
# elimino las filas de los locales sin reseñas
df_pizza_CANV_rev1 = df_pizza_CANV_rev[df_pizza_CANV_rev['user_id'].notna()]

In [41]:
df_pizza_CANV_rev1.head()

,gmap_id,business_name,address,city,state,zip_code,latitude,longitude,avg_rating,num_of_reviews,user_id,user_name,rating,text,date
0,0x80c2c7726b53b8e5:0xad288d11654f5e48,PIZZA ROW RECORDS,433 E 7th St,Los Angeles,CA,90014,34.041406,-118.247477,4.3,34,1.159800e+20,Patrick Sutter,5.0,"Oh man, this little pizzeria carries the best ...",2019-08-10 16:42:31.213
1,0x80c2c7726b53b8e5:0xad288d11654f5e48,PIZZA ROW RECORDS,433 E 7th St,Los Angeles,CA,90014,34.041406,-118.247477,4.3,34,1.062145e+20,Branden Montgomery,4.0,Excellent service and food 🍕😋,2021-08-22 21:24:59.592
2,0x80c2c7726b53b8e5:0xad288d11654f5e48,PIZZA ROW RECORDS,433 E 7th St,Los Angeles,CA,90014,34.041406,-118.247477,4.3,34,1.029614e+20,Bubba Barks,4.0,"I'm from NYC, so I would call myself a pizza 🍕...",2019-09-13 07:05:09.213
3,0x80c2c7726b53b8e5:0xad288d11654f5e48,PIZZA ROW RECORDS,433 E 7th St,Los Angeles,CA,90014,34.041406,-118.247477,4.3,34,1.089620e+20,Kathryn Nguyen,5.0,"That is some good pizza, and very tasty wings ...",2019-11-28 07:44:09.374
4,0x80c2c7726b53b8e5:0xad288d11654f5e48,PIZZA ROW RECORDS,433 E 7th St,Los Angeles,CA,90014,34.041406,-118.247477,4.3,34,1.172350e+20,Jose Hernandez,5.0,This place is awesome great service from the o...,2019-08-25 14:23:22.205


In [42]:
# cuento la cantidad de reseñas de pizzerias en cada estado / no puedo contar con una sola columna
df_pizza_CANV_rev1[['state', 'gmap_id' ]].groupby('state').count()

,gmap_id
state,
CA,27574
NV,7331


Hay 27574 reseñas para locales del estado de CA y 7331 reseñas para locales del estado de NV

In [43]:
# guardo df_pizza_NJNY_rev
df_pizza_CANV_rev1.to_parquet(r'C:\Users\gonza\OneDrive\Desktop\Proyecto Grupal Henry\Maps\df_pizza_CANV_rev.parquet' , engine='fastparquet')